In [94]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.table import Table
import numpy as np
import seaborn as sns
from io import StringIO

**Data Profiling**

Data profiling is the process of analyzing and understanding the characteristics and structure of a dataset. It involves examining the data to identify patterns, anomalies, and inconsistencies, as well as determining the quality and completeness of the data.

Data profiling typically involves looking at the basic statistics of the dataset, such as the number of observations, the number of missing values, the distribution of values, and the presence of outliers. 


In [95]:
df_org = pd.read_csv("homepap_merged dataset.csv")
print(df.columns)
df = df_org

Index(['nsrrid', 'age', 'gender', 'race3', 'ethnicity', 'neckcm', 'bmi',
       'systolic', 'diastolic', 'dxdiab', 'dxhtn', 'dxdep', 'dxemph', 'ahi',
       'acceptance', 'cal_total', 'ahi_full', 'fosq_global', 'systolic.1',
       'diastolic.1', 'cal_total.1', 'fosq_global.1', 'avgpapuse.1', 'bmi.3',
       'systolic.3', 'diastolic.3', 'cal_total.3', 'fosq_global.3',
       'avgpapuse.3'],
      dtype='object')


In the code below, I have analysed the basic stats of the baseline dataset mainly column age and gender. I have also calculated the number of unique and null values in each column. And to understand the distribution better, I have used histogram. Skewness of the data is also printed. 

In [96]:

summary_stats = df.describe().loc[['count', 'mean', 'min','max']].round(2).transpose()

null_count = df.isnull().sum()
skewness = df.skew()

summary_df = pd.concat([summary_stats, null_count.to_frame(name='null count'), skewness.to_frame(name='skewness')], axis=1)

print(summary_df)

               count        mean         min         max  null count  skewness
nsrrid         142.0  1600188.27  1600001.00  1600373.00           0 -0.061840
age            142.0       49.46       21.00       80.00           0 -0.063849
gender         142.0        0.65        0.00        1.00           0 -0.625887
race3          142.0        1.34        1.00        3.00           0  1.698415
ethnicity      141.0        1.91        1.00        2.00           1 -2.849574
neckcm         142.0       43.86       34.67       56.07           0  0.070078
bmi            141.0       38.78       23.59       71.48           1  1.048987
systolic       141.0      127.27       97.00      165.00           1  0.396164
diastolic      141.0       79.71       56.00      106.00           1 -0.013167
dxdiab         141.0        0.21        0.00        1.00           1  1.472071
dxhtn          140.0        0.58        0.00        1.00           2 -0.321697
dxdep          137.0        0.25        0.00        


Data validation
1. nsrrid: Check for duplicates or invalid identifiers.
2. age: Check for negative values or unrealistic age values (e.g., > 120).
3. gender: Check for consistency in categorical representation (e.g., 0/1 or Male/Female) and remove any invalid entries.
4. race3: Validate the categorical representation, and remove or impute any invalid entries.
5. ethnicity: Validate the categorical representation, and remove or impute any invalid entries.
6. neckcm: Check for negative or unrealistic values.
7. bmi: Check for negative or unrealistic values.
8. systolic: Check for negative or unrealistic values.
9. diastolic: Check for negative or unrealistic values.
10. dxdiab: Validate the categorical representation, and remove or impute any invalid entries.
11. dxhtn: Validate the categorical representation, and remove or impute any invalid entries.
12. dxdep: Validate the categorical representation, and remove or impute any invalid entries.
13. dxemph: Validate the categorical representation, and remove or impute any invalid entries.
14. ahi: Check for negative or unrealistic values.
15. acceptance: Validate the categorical representation, and remove or impute any invalid entries.
16. cal_total: Check for negative or unrealistic values.
17. ahi_full: Check for negative or unrealistic values.
18. fosq_global: Check for negative or unrealistic values.

Data Cleansing

Data cleansing, also known as data cleaning or data scrubbing, is the process of identifying and correcting or removing errors, inconsistencies, and inaccuracies in a dataset. The goal of data cleansing is to improve the quality of the data, making it more accurate, complete, and consistent.


In [97]:
df = df.drop_duplicates(subset='nsrrid', keep='first')
df = df[(df['age'] >= 0) & (df['age'] <= 120)]
gender_mapping = {'Male': 0, 'Female': 1}
df['gender'] = df['gender'].replace(gender_mapping)
df = df[(df['neckcm'] > 0) ]
df = df[(df['bmi.3'] > 0) | (df['bmi'] > 0)]

df = df[(df['systolic'] > 0) & (df['diastolic'] > 0)]
df = df[(df['systolic.1'] > 0) & (df['diastolic.1'] > 0)]
df = df[(df['systolic.3'] > 0) & (df['diastolic.3'] > 0)]


binary_columns = ['dxdiab', 'dxhtn', 'dxdep', 'dxemph']
for col in binary_columns:
    df = df[df[col].isin([0, 1])]
df = df[df['ahi'] >= 0]
df = df[df['acceptance'].isin([0, 1])]
df = df[ (df['cal_total'] >= 0) | (df['cal_total.1'] >= 0) | (df['cal_total.3'] >= 0)]

df = df[ (df['fosq_global'] >= 0) | (df['fosq_global.1'] >= 0) | (df['fosq_global.3'] >= 0)]

 
df = df[(df['avgpapuse.1'] >= 0) | (df['avgpapuse.3'] >= 0)]

df


,nsrrid,age,gender,race3,ethnicity,neckcm,bmi,systolic,diastolic,dxdiab,...,diastolic.1,cal_total.1,fosq_global.1,avgpapuse.1,bmi.3,systolic.3,diastolic.3,cal_total.3,fosq_global.3,avgpapuse.3
0,1600001,52,1,2,2.0,43.166667,33.234794,124.0,81.0,0.0,...,71.0,4.88,19.75,268.0,32.637582,130.0,65.0,4.87,19.78,206.0
1,1600002,53,1,1,2.0,50.266667,40.818115,125.0,77.0,0.0,...,84.0,3.07,16.74,54.0,39.883126,140.0,81.0,4.34,18.19,243.0
2,1600007,58,1,3,2.0,47.066667,46.909469,128.0,61.0,1.0,...,60.0,2.46,12.45,369.0,45.327784,121.0,64.0,1.76,15.90,352.0
4,1600010,36,1,1,2.0,48.366667,35.662224,133.0,87.0,0.0,...,77.0,4.59,18.13,130.0,35.297793,150.0,89.0,2.61,11.32,29.0
5,1600011,22,1,1,2.0,43.000000,34.772522,132.0,84.0,0.0,...,76.0,5.32,19.67,141.0,34.659348,120.0,76.5,4.70,19.10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,1600365,32,1,2,2.0,48.166667,52.532243,132.0,84.0,0.0,...,89.0,4.96,17.75,120.0,51.617147,175.0,100.0,4.79,19.75,199.0
138,1600366,34,1,1,2.0,42.333333,28.650791,121.0,81.0,0.0,...,85.0,2.42,15.85,55.0,29.150151,130.0,90.0,2.53,15.76,NaN
139,1600370,51,0,2,2.0,43.133333,37.633309,159.0,79.0,1.0,...,81.0,2.90,16.75,95.0,37.370343,134.0,77.0,3.98,16.64,264.0
140,1600372,62,1,1,2.0,50.766667,36.571429,122.0,78.0,0.0,...,72.0,6.19,19.49,260.0,36.865306,132.0,68.0,6.59,19.60,193.0


In [98]:
df_clean = df.dropna(subset=['fosq_global'])

print(f"Original number of rows: {len(df_org)}")
print(f"Number of rows after removing missing values: {len(df_clean)}")


Original number of rows: 142
Number of rows after removing missing values: 121
